In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [55]:
from scipy import stats

In [2]:
data = pd.read_csv('water.txt',sep='\t',header=0)

In [3]:
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [4]:
data[['mortality','hardness']].corr()# 1 вопрос OK

,mortality,hardness
mortality,1.000000,-0.654849
hardness,-0.654849,1.000000


In [6]:
data[['mortality','hardness']].corr(method='spearman')# 2 вопрос OK

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


In [11]:
data[data.location=='North'][['mortality','hardness']].corr()# 3 овопрос OK

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


In [12]:
data[data.location=='South'][['mortality','hardness']].corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [118]:
bar_data = pd.DataFrame([[203,718],[239,515]],dtype=np.float)

In [119]:
bar_data.columns=['NoDrink','Drink']

In [120]:
bar_data.index = ['Female','Male']

In [121]:
bar_data

,NoDrink,Drink
Female,203.0,718.0
Male,239.0,515.0


In [53]:
def MMC(x): 
    TP = x.ix[0,0]
    TN = x.ix[1,1]
    FN = x.ix[0,1]
    FP = x.ix[1,0]
    #print TP,TN,FN,FP
    return (TP*TN - FP*FN)/ np.sqrt((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))

In [122]:
MMC(bar_data) # 4 вопрос OK

-0.10900237458678963

In [128]:
stat,p,__,__ = stats.chi2_contingency(bar_data)# 5 вопрос OK
p

1.0558987006638725e-05

In [68]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [125]:
p_female = [1]*int(bar_data.ix[0,0]) + [0]*int(bar_data.ix[0,1])
p_male = [1]*int(bar_data.ix[1,0]) + [0]*int(bar_data.ix[1,1])

In [126]:
proportions_confint_diff_ind(p_male,p_female)# 6 ответ OK

(0.053905233215813156, 0.13922183141523897)

In [70]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [73]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [129]:
proportions_diff_z_test(proportions_diff_z_stat_ind(p_male,p_female))# 7 вопрос OK

8.1534530895766011e-06

In [84]:
arr = np.array([[197,111,33],
                [382,685,331],
                [110,342,333]],dtype=np.float)

In [85]:
last_data = pd.DataFrame(arr,columns=['Не доволен','Более или менее','Доволен'],
             index=['Не очень счастлив','Достаточно счастлив','Очень счастлив'])
last_data

,Не доволен,Более или менее,Доволен
Не очень счастлив,197.0,111.0,33.0
Достаточно счастлив,382.0,685.0,331.0
Очень счастлив,110.0,342.0,333.0


In [145]:
def corr_chi2_stat(arr):
    n = arr.sum()
    sum = 0.0
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            v = (arr[i,:].sum()*arr[:,j].sum()) / n
            sum += ( (arr[i,j] - v)**2 / v )
    return sum

In [146]:
corr_chi2_stat(arr)# 8 вопрос OK

293.68311039689752

In [149]:
stats.chisqprob(corr_chi2_stat(arr),1)# 9 вопрос

7.8353253558532578e-66

In [156]:
stats.chi2_contingency(arr)# 9 вопрос OK

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[  93.08597464,  153.74722662,   94.16679873],
        [ 381.6251981 ,  630.318542  ,  386.0562599 ],
        [ 214.28882726,  353.93423138,  216.77694136]]))

In [151]:
# 10 вопрос OK
v_kramer = np.sqrt(corr_chi2_stat(arr) / ( arr.sum() * (min(arr.shape[0],arr.shape[1]) -1.0 ) ) )
v_kramer

0.24120139345003383